# Science Plan

Objective - test hypothesis that weather events can affect the bottom currents that control the bending and rise heights of Hydrothermal plumes 

Mostly copied from the New Sacker Science Plan notebook, however this notebook reduces what is present to only the most important data and figures for the poster. Many of the missing figures can be found in the New Sacker Science Plan notebook for reference. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits import mplot3d
import pandas as pd
from datetime import datetime
import seaborn as sns
import hvplot.pandas
import holoviews as hv
from holoviews import dim, opts
hv.extension('bokeh')

## Step One - Find Some Data

* Check out the loaded zip file. It contains plume bending data (direction of bending and magnitude of bending), vertical velocity in the plume, a vent temperature data file, and some weather data.  Note these files contain data for two time frames - Oct 2010 and Oct to Dec 2011. 
    i. BendData*.txt - these two files are the basic bending data - three columns = {direction as angle from north, bending magnitude as angle from vertical, julian date}
    ii. Other files are .mat format so Dax or I may need to help with these. I’ll try to load more information on them soon.
* I also loaded two powerpoints of talks that came out of the pilot study.  Some of the material is irrelevant.
* I don’t have a handy tidal data file -- bottom current and pressure from tides -- but this should exist at least as model data
* At Ocean Networks Canada’s NEPTUNE observatory, they had current meters (ADCP) at a regional circulation mooring about 1 km N to NE of the COVIS site that collected data in Oct 2010 and in late 2011 to early 2012. 
* I did include the weather data I found. This came from the NOAA and National Weather Center’s records.  Feel free to do your own hunt for data!
* At some point, you might want the actual grids of COVIS data.  Right now most data is in Matlab’s .mat format and takes a bit of processing to get images, centerlines, and bending data.  But this will be useful to lengthen the data series (COVIS took data in Oct 2010 and form Oct 2011 to some time in late 2014 or early 2015).

**New bending data (angles.partialpts_2010/2011.dat) have replaced the BendData files. This bending data as initially ingested contains two angles:**

**1. inclination - which is the angle from the vertical (positive z-axis)**

**2. azimuth - which is the counterclockwise angle from the postive x-axis (standard for polar coordinate systems)**

## Recomputed Plume Bending Data 

**2010**

In [ ]:
!echo 'date,inclination,azimuth' | cat - ~jovyan/data/covis_data/recomputed/angles_partialpts_2010.dat > ~jovyan/data/covis_data/temp.txt
!sed 's/ /,/g' ~jovyan/data/covis_data/temp.txt > ~jovyan/data/covis_data/recomputed/angles_partialpts_2010.csv
!rm ~jovyan/data/covis_data/temp*
!head /home/jovyan/data/covis_data/recomputed/angles_partialpts_2010.csv

In [ ]:
path = '/home/jovyan/data/covis_data/recomputed/angles_partialpts_2010.csv' 
df_rc2010 = pd.read_csv(path, sep=",")
df_rc2010['year'] = '2010'
df_rc2010['datetime'] = pd.to_datetime(df_rc2010.year, format='%Y') + pd.to_timedelta(df_rc2010.date - 1, unit='d')
df_rc2010['datetime'] = df_rc2010['datetime'].dt.round('1s')
df_rc2010 = df_rc2010.set_index('datetime')
df_rc2010.drop(['date', 'year'], axis=1,inplace=True)
df_rc2010 = df_rc2010.rename_axis(None)
#fix for azimuth angle, 90 - azimuth then recomputation for any values that aren't in between 0 and 360 
df_rc2010.azimuth = df_rc2010.azimuth.rsub(90)
df_rc2010['azimuth'] %= 360 
df_rc2010 = df_rc2010.resample('h').mean()
df_rc2010.head(20)

In [ ]:
#not resampled version (nr)
path = '/home/jovyan/data/covis_data/recomputed/angles_partialpts_2010.csv' 
df_nrrc2010 = pd.read_csv(path, sep=",")
df_nrrc2010['year'] = '2010'
df_nrrc2010['datetime'] = pd.to_datetime(df_nrrc2010.year, format='%Y') + pd.to_timedelta(df_nrrc2010.date - 1, unit='d')
df_nrrc2010['datetime'] = df_nrrc2010['datetime'].dt.round('1s')
df_nrrc2010 = df_nrrc2010.set_index('datetime')
df_nrrc2010.drop(['date', 'year'], axis=1,inplace=True)
df_nrrc2010 = df_nrrc2010.rename_axis(None)
#fix for azimuth angle, 90 - azimuth then recomputation for any values that aren't in between 0 and 360 
df_nrrc2010.azimuth = df_nrrc2010.azimuth.rsub(90)
df_nrrc2010['azimuth'] %= 360 
df_nrrc2010.head(20)

**2011**

In [ ]:
!echo 'date,inclination,azimuth' | cat - ~jovyan/data/covis_data/recomputed/angles_partialpts_2011.dat > ~jovyan/data/covis_data/temp.txt
!sed 's/ /,/g' ~jovyan/data/covis_data/temp.txt > ~jovyan/data/covis_data/recomputed/angles_partialpts_2011.csv
!rm ~jovyan/data/covis_data/temp*
!head /home/jovyan/data/covis_data/recomputed/angles_partialpts_2011.csv

In [ ]:
path = '/home/jovyan/data/covis_data/recomputed/angles_partialpts_2011.csv' 
df_rc2011 = pd.read_csv(path, sep=",")
df_rc2011['year'] = '2011'
df_rc2011['datetime'] = pd.to_datetime(df_rc2011.year, format='%Y') + pd.to_timedelta(df_rc2011.date - 1, unit='d')
df_rc2011['datetime'] = df_rc2011['datetime'].dt.round('1s')
df_rc2011 = df_rc2011.set_index('datetime')
df_rc2011.drop(['date', 'year'], axis=1,inplace=True)
df_rc2011 = df_rc2011.rename_axis(None)
#fix for azimuth angle, 90 - azimuth then recomputation for any values that aren't in between 0 and 360
df_rc2011.azimuth = df_rc2011.azimuth.rsub(90)
df_rc2011['azimuth'] %= 360 
df_rc2011 = df_rc2011.resample('h').mean()
df_rc2011.head(3)

In [ ]:
#not resampled version (nr)
path = '/home/jovyan/data/covis_data/recomputed/angles_partialpts_2011.csv' 
df_nrrc2011 = pd.read_csv(path, sep=",")
df_nrrc2011['year'] = '2011'
df_nrrc2011['datetime'] = pd.to_datetime(df_nrrc2011.year, format='%Y') + pd.to_timedelta(df_nrrc2011.date - 1, unit='d')
df_nrrc2011['datetime'] = df_nrrc2011['datetime'].dt.round('1s')
df_nrrc2011 = df_nrrc2011.set_index('datetime')
df_nrrc2011.drop(['date', 'year'], axis=1,inplace=True)
#fix for azimuth angle, 90 - azimuth then recomputation for any values that aren't in between 0 and 360
df_nrrc2011 = df_nrrc2011.rename_axis(None)
df_nrrc2011.azimuth = df_nrrc2011.azimuth.rsub(90)
df_nrrc2011['azimuth'] %= 360 
df_nrrc2011.head(3)

## Weather Data

(weather_data_for_plotting*.txt files)

 An explanation of the variable names can be found at https://www.ndbc.noaa.gov/measdes.shtml

13 variables (Year, Month, Day, Hour, Minute, Seconds, Julian day, wave height, wind direction, wind speed, wind gust speed, atmospheric pressure, air temperature)

separator = single blank space

Notes:
1. Time is given both as a vector (from original data file probably) and as a Julian day.
2. No information on other units by m/s likely for speeds.  Rest should match the NOAA information.  
3. I probably still have the original NOAA data files for these buoys. I'll see if I can match data files to the time periods; my directories seemed a little confused. 

**2010 C46036**

In [ ]:
!rm ~jovyan/data/covis_data/weather_data_for_plotting_2010_C46036.csv
!echo 'year,month,day,hour,minute,seconds,jday,wv_height,wnd_dir,wnd_spd,wnd_gspd,atm_prs,air_temp' | cat - ~jovyan/data/covis_data/weather_data_for_plotting_2010_C46036.txt > ~jovyan/data/covis_data/temp.txt
!sed 's/   /,/g' ~jovyan/data/covis_data/temp.txt > ~jovyan/data/covis_data/temp1.txt
!sed 's/^,//' <~jovyan/data/covis_data/temp1.txt >~jovyan/data/covis_data/weather_data_for_plotting_2010_C46036.csv
!rm ~jovyan/data/covis_data/temp*

In [ ]:
path = '/home/jovyan/data/covis_data/weather_data_for_plotting_2010_C46036.csv' 
df_wd2010c46036 = pd.read_csv(path, sep=",")
df_wd2010c46036['year']= df_wd2010c46036['year'].astype(int).astype(str)
df_wd2010c46036['month']= df_wd2010c46036['month'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2010c46036['day']= df_wd2010c46036['day'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2010c46036['hour']= df_wd2010c46036['hour'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2010c46036['minute']=df_wd2010c46036['minute'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2010c46036['datetime'] = df_wd2010c46036['year'] + df_wd2010c46036['month'] + df_wd2010c46036['day'] +\
'T' + df_wd2010c46036['hour']+ ':' + df_wd2010c46036['minute']
df_wd2010c46036['datetime'] = pd.to_datetime(df_wd2010c46036['datetime'])
df_wd2010c46036 = df_wd2010c46036.set_index('datetime')
df_wd2010c46036 = df_wd2010c46036[['jday', 'wv_height', 'wnd_dir', 'wnd_spd', 'wnd_gspd', 'atm_prs', 'air_temp']]
df_wd2010c46036 = df_wd2010c46036.rename_axis(None)
df_wd2010c46036 = df_wd2010c46036.resample('h').mean()
df_wd2010c46036

**2011 C46036**

In [ ]:
!rm ~jovyan/data/covis_data/weather_data_for_plotting_2011_C46036.csv
!echo 'year,month,day,hour,minute,seconds,jday,wv_height,wnd_dir,wnd_spd,wnd_gspd,atm_prs,air_temp' | cat - ~jovyan/data/covis_data/weather_data_for_plotting_2011_C46036.txt > ~jovyan/data/covis_data/temp.txt
!sed 's/   /,/g' ~jovyan/data/covis_data/temp.txt > ~jovyan/data/covis_data/temp1.txt
!sed 's/^,//' <~jovyan/data/covis_data/temp1.txt >~jovyan/data/covis_data/weather_data_for_plotting_2011_C46036.csv
!rm ~jovyan/data/covis_data/temp*

In [ ]:
path = '/home/jovyan/data/covis_data/weather_data_for_plotting_2011_C46036.csv'
df_wd2011c46036 = pd.read_csv(path, sep=",")
df_wd2011c46036['year']= df_wd2011c46036['year'].astype(int).astype(str)
df_wd2011c46036['month']= df_wd2011c46036['month'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2011c46036['day']= df_wd2011c46036['day'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2011c46036['hour']= df_wd2011c46036['hour'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2011c46036['minute']=df_wd2011c46036['minute'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2011c46036['datetime'] = df_wd2011c46036['year'] + df_wd2011c46036['month'] + df_wd2011c46036['day'] +\
'T' + df_wd2011c46036['hour']+ ':' + df_wd2011c46036['minute']
df_wd2011c46036['datetime'] = pd.to_datetime(df_wd2011c46036['datetime'])
df_wd2011c46036 = df_wd2011c46036.set_index('datetime')
df_wd2011c46036 = df_wd2011c46036[['jday', 'wv_height', 'wnd_dir', 'wnd_spd', 'wnd_gspd', 'atm_prs', 'air_temp']]
df_wd2011c46036 = df_wd2011c46036.rename_axis(None)
df_wd2011c46036 = df_wd2011c46036.resample('h').mean()
df_wd2011c46036

**2010 Tillamook**

In [ ]:
!head ~jovyan/data/covis_data/weather_data_for_plotting_2010_Tillamook.txt

In [ ]:
!rm ~jovyan/data/covis_data/weather_data_for_plotting_2010_Tillamook.csv
!echo 'year,month,day,hour,minute,seconds,jday,wv_height,wnd_dir,wnd_spd,wnd_gspd,atm_prs,air_temp' | cat - ~jovyan/data/covis_data/weather_data_for_plotting_2010_Tillamook.txt > ~jovyan/data/covis_data/temp.txt
!sed 's/   /,/g' ~jovyan/data/covis_data/temp.txt > ~jovyan/data/covis_data/temp1.txt
!sed 's/,,,,/,/g' ~jovyan/data/covis_data/temp1.txt > ~jovyan/data/covis_data/temp2.txt
!sed 's/^,//' <~jovyan/data/covis_data/temp2.txt >~jovyan/data/covis_data/weather_data_for_plotting_2010_Tillamook.csv
!rm ~jovyan/data/covis_data/temp*
!head ~jovyan/data/covis_data/weather_data_for_plotting_2010_Tillamook.csv

In [ ]:
path = '/home/jovyan/data/covis_data/weather_data_for_plotting_2010_Tillamook.csv' 
df_wd2010Tillamook = pd.read_csv(path, sep=",")
df_wd2010Tillamook['year']= df_wd2010Tillamook['year'].astype(int).astype(str)
df_wd2010Tillamook['month']= df_wd2010Tillamook['month'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2010Tillamook['day']= df_wd2010Tillamook['day'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2010Tillamook['hour']= df_wd2010Tillamook['hour'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2010Tillamook['minute']=df_wd2010Tillamook['minute'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2010Tillamook['wv_height'] = df_wd2010Tillamook['wv_height'].astype('float64')
df_wd2010Tillamook['wnd_dir'] = df_wd2010Tillamook['wnd_dir'].astype('float64')
df_wd2010Tillamook['wnd_spd'] = df_wd2010Tillamook['wnd_spd'].astype('float64')
df_wd2010Tillamook['wnd_gspd'] = df_wd2010Tillamook['wnd_gspd'].astype('float64')
df_wd2010Tillamook['atm_prs'] = df_wd2010Tillamook['atm_prs'].astype('float64')
df_wd2010Tillamook['air_temp'] = df_wd2010Tillamook['air_temp'].astype('float64')
df_wd2010Tillamook['datetime'] = df_wd2010Tillamook['year'] + df_wd2010Tillamook['month'] + df_wd2010Tillamook['day'] +\
'T' + df_wd2010Tillamook['hour']+ ':' + df_wd2010Tillamook['minute']
df_wd2010Tillamook['datetime'] = pd.to_datetime(df_wd2010Tillamook['datetime'])
df_wd2010Tillamook = df_wd2010Tillamook.set_index('datetime')
df_wd2010Tillamook = df_wd2010Tillamook[['jday', 'wv_height', 'wnd_dir', 'wnd_spd', 'wnd_gspd', 'atm_prs', 'air_temp']]
df_wd2010Tillamook = df_wd2010Tillamook.rename_axis(None)
df_wd2010Tillamook = df_wd2010Tillamook.resample('h').mean()
df_wd2010Tillamook

**2011 Tillamook**

In [ ]:
!head ~jovyan/data/covis_data/weather_data_for_plotting_2011_Tillamook.txt

In [ ]:
!rm ~jovyan/data/covis_data/weather_data_for_plotting_2011_Tillamook.csv
!echo 'year,month,day,hour,minute,seconds,jday,wv_height,wnd_dir,wnd_spd,wnd_gspd,atm_prs,air_temp' | cat - ~jovyan/data/covis_data/weather_data_for_plotting_2011_Tillamook.txt > ~jovyan/data/covis_data/temp.txt
!sed 's/   /,/g' ~jovyan/data/covis_data/temp.txt > ~jovyan/data/covis_data/temp1.txt
!sed 's/,,,,/,/g' ~jovyan/data/covis_data/temp1.txt > ~jovyan/data/covis_data/temp2.txt
!sed 's/  /,/g' ~jovyan/data/covis_data/temp2.txt > ~jovyan/data/covis_data/temp3.txt
!sed 's/^,//' <~jovyan/data/covis_data/temp3.txt >~jovyan/data/covis_data/weather_data_for_plotting_2011_Tillamook.csv
!rm ~jovyan/data/covis_data/temp*
!head ~jovyan/data/covis_data/weather_data_for_plotting_2011_Tillamook.csv

In [ ]:
path = '/home/jovyan/data/covis_data/weather_data_for_plotting_2011_Tillamook.csv' 
df_wd2011Tillamook = pd.read_csv(path, sep=",", engine='python')
df_wd2011Tillamook= df_wd2011Tillamook.dropna()
df_wd2011Tillamook['year']= df_wd2011Tillamook['year'].astype(int).astype(str)
df_wd2011Tillamook['month']= df_wd2011Tillamook['month'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2011Tillamook['day']= df_wd2011Tillamook['day'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2011Tillamook['hour']= df_wd2011Tillamook['hour'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2011Tillamook['minute']=df_wd2011Tillamook['minute'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2011Tillamook['wv_height'] = df_wd2011Tillamook['wv_height'].astype('float64')
df_wd2011Tillamook['wnd_dir'] = df_wd2011Tillamook['wnd_dir'].astype('float64')
df_wd2011Tillamook['wnd_spd'] = df_wd2011Tillamook['wnd_spd'].astype('float64')
df_wd2011Tillamook['wnd_gspd'] = df_wd2011Tillamook['wnd_gspd'].astype('float64')
df_wd2011Tillamook['atm_prs'] = df_wd2011Tillamook['atm_prs'].astype('float64')
df_wd2011Tillamook['air_temp'] = df_wd2011Tillamook['air_temp'].astype('float64')
df_wd2011Tillamook['datetime'] = df_wd2011Tillamook['year'] + df_wd2011Tillamook['month'] + df_wd2011Tillamook['day'] +\
'T' + df_wd2011Tillamook['hour']+ ':' + df_wd2011Tillamook['minute']
df_wd2011Tillamook['datetime'] = pd.to_datetime(df_wd2011Tillamook['datetime'])
df_wd2011Tillamook = df_wd2011Tillamook.set_index('datetime')
df_wd2011Tillamook = df_wd2011Tillamook[['jday', 'wv_height', 'wnd_dir', 'wnd_spd', 'wnd_gspd', 'atm_prs', 'air_temp']]
df_wd2011Tillamook = df_wd2011Tillamook.rename_axis(None)
df_wd2011Tillamook = df_wd2011Tillamook.resample('h').mean()
df_wd2011Tillamook.head()

## **2010**

## Recomputed Plume Bending Data and C46036 Weather Data

In [ ]:
rc2010c = pd.merge(df_wd2010c46036, df_rc2010,how='inner', indicator=True, left_index=True, right_index=True, suffixes=('_B', '_G'))

In [ ]:
df_covis_rc2010c = pd.DataFrame()
df_covis_rc2010c = rc2010c[rc2010c['_merge'] == 'both']
del df_covis_rc2010c['_merge']
df_covis_rc2010c = df_covis_rc2010c.dropna()
del df_covis_rc2010c['jday']
df_covis_rc2010c.head()

In [ ]:
df_covis_rc2010c.describe()

**Histograms**

In [ ]:
df_covis_rc2010c.hvplot.hist(y='azimuth')

In [ ]:
degrees = df_covis_rc2010c['azimuth'].values
radians = np.deg2rad(degrees)

bin_size = 2
a , b=np.histogram(degrees, bins=np.arange(0, 360+bin_size, bin_size))
centers = np.deg2rad(np.ediff1d(b)//2 + b[:-1])

fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection='polar')
ax.bar(centers, a, width=np.deg2rad(bin_size), bottom=0.0, color='cyan', edgecolor='k')
ax.set_theta_zero_location("N")
ax.set_theta_direction(-1)
plt.show()

In [ ]:
df_covis_rc2010c['inclination_category'] = pd.cut(df_covis_rc2010c['inclination'], bins=[0, 10, 40, 60], include_lowest=True, labels=['vertical', 'ordinary', 'extreme'])
df_covis_rc2010c.hvplot.hist(y='inclination', by='inclination_category')

**Violin Plots**

In [ ]:
df_covis_rc2010c.inclination.hvplot.violin(by='index.day')

In [ ]:
day2010c=df_covis_rc2010c.index.day
i2010c=df_covis_rc2010c['inclination']
sns.violinplot(day2010c, i2010c)

In [ ]:
plt.violinplot(i2010c, showmeans=True, showextrema=True, showmedians=False)

**Cross Correlation**

In [ ]:
df_covis_rc2010c.corr()

## Recomputed Plume Bending Data and Tillamook Weather Data

***Dropna() command results in no dataframe being shown***

In [ ]:
rc2010T = pd.merge(df_wd2010Tillamook, df_rc2010,how='inner', indicator=True, left_index=True, right_index=True, suffixes=('_B', '_G'))

In [ ]:
df_covis_rc2010T = pd.DataFrame()
df_covis_rc2010T = rc2010T[rc2010T['_merge'] == 'both']
del df_covis_rc2010T['_merge']
#df_covis_rc2010c = df_covis_rc2010c.dropna(axis=1,how='all')
#df_covis_rc2010T = df_covis_rc2010T.dropna()
del df_covis_rc2010T['jday']
df_covis_rc2010T.head(20)

In [ ]:
df_covis_rc2010T.describe()

**Histograms**

In [ ]:
df_covis_rc2010T.hvplot.hist(y='azimuth')

In [ ]:
degrees = df_covis_rc2010T['azimuth'].values
radians = np.deg2rad(degrees)

bin_size = 2
a , b=np.histogram(degrees, bins=np.arange(0, 360+bin_size, bin_size))
centers = np.deg2rad(np.ediff1d(b)//2 + b[:-1])

fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection='polar')
ax.bar(centers, a, width=np.deg2rad(bin_size), bottom=0.0, color='cyan', edgecolor='k')
ax.set_theta_zero_location("N")
ax.set_theta_direction(-1)
plt.show()

In [ ]:
df_covis_rc2010T['inclination_category'] = pd.cut(df_covis_rc2010T['inclination'], bins=[0, 10, 40, 60], include_lowest=True, labels=['vertical', 'ordinary', 'extreme'])
df_covis_rc2010T.hvplot.hist(y='inclination', by='inclination_category')

***Violin plots are the same from above (2010 C46036)***

**Cross Correlation**

In [ ]:
df_covis_rc2010T.corr()

## **2011**

## Recomputed Plume Bending Data and C46036 Weather Data

In [ ]:
rc2011c = pd.merge(df_wd2011c46036, df_rc2011,how='inner', indicator=True, left_index=True, right_index=True, suffixes=('_B', '_G'))

In [ ]:
df_covis_rc2011c = pd.DataFrame()
df_covis_rc2011c = rc2011c[rc2011c['_merge'] == 'both']
del df_covis_rc2011c['_merge']
df_covis_rc2011c = df_covis_rc2011c.dropna()
del df_covis_rc2011c['jday']
df_covis_rc2011c.head()

In [ ]:
df_covis_rc2011c.describe()

**Histograms**

In [ ]:
df_covis_rc2011c.hvplot.hist(y='azimuth')

In [ ]:
degrees = df_covis_rc2011c['azimuth'].values
radians = np.deg2rad(degrees)

bin_size = 2
a , b=np.histogram(degrees, bins=np.arange(0, 360+bin_size, bin_size))
centers = np.deg2rad(np.ediff1d(b)//2 + b[:-1])

fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection='polar')
ax.bar(centers, a, width=np.deg2rad(bin_size), bottom=0.0, color='cyan', edgecolor='k')
ax.set_theta_zero_location("N")
ax.set_theta_direction(-1)
plt.show()

In [ ]:
df_covis_rc2011c['inclination_category'] = pd.cut(df_covis_rc2011c['inclination'], bins=[0, 10, 40, 60], include_lowest=True, labels=['vertical', 'ordinary', 'extreme'])
df_covis_rc2011c.hvplot.hist(y='inclination', by='inclination_category')

**Violin Plots**

In [ ]:
df_covis_rc2011c.inclination.hvplot.violin(by='index.day')

In [ ]:
day2011c=df_covis_rc2011c.index.day
i2011c=df_covis_rc2011c['inclination']
sns.violinplot(day2011c, i2011c)

In [ ]:
plt.violinplot(i2011c, showmeans=True, showextrema=True, showmedians=False)

**Cross Correlation**

In [ ]:
df_covis_rc2011c.corr()

## Recomputed Plume Bending Data and Tillamook Weather Data

***Dropna() command results in no dataframe being shown***

In [ ]:
rc2011T = pd.merge(df_wd2011Tillamook, df_rc2011,how='inner', indicator=True, left_index=True, right_index=True, suffixes=('_B', '_G'))

In [ ]:
df_covis_rc2011T = pd.DataFrame()
df_covis_rc2011T = rc2011T[rc2011T['_merge'] == 'both']
del df_covis_rc2011T['_merge']
#df_covis_rc2011T = df_covis_rc2011T.dropna(axis=1,how='all')
#df_covis_rc2011T = df_covis_rc2011T.dropna()
del df_covis_rc2011T['jday']
df_covis_rc2011T.head()

In [ ]:
df_covis_rc2011T.describe()

**Histograms**

In [ ]:
df_covis_rc2011T.hvplot.hist(y='azimuth')

In [ ]:
degrees = df_covis_rc2011T['azimuth'].values
radians = np.deg2rad(degrees)

bin_size = 2
a , b=np.histogram(degrees, bins=np.arange(0, 360+bin_size, bin_size))
centers = np.deg2rad(np.ediff1d(b)//2 + b[:-1])

fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection='polar')
ax.bar(centers, a, width=np.deg2rad(bin_size), bottom=0.0, color='cyan', edgecolor='k')
ax.set_theta_zero_location("N")
ax.set_theta_direction(-1)
plt.show()

In [ ]:
df_covis_rc2011T['inclination_category'] = pd.cut(df_covis_rc2011T['inclination'], bins=[0, 10, 40, 60], include_lowest=True, labels=['vertical', 'ordinary', 'extreme']) 
df_covis_rc2011T.hvplot.hist(y='inclination', by='inclination_category')

***Violin plots are the same from above (2011 C46036)***

**Cross Correlation**

In [ ]:
df_covis_rc2011T.corr()

## Step Two - Plot The Data
* What patterns do you see?  
* What else can you do with this data?
* Do the different data sets correlate?


## **2010 and 2011 Plume Bending Data Histograms**

**Azimuth**

In [ ]:
#histogram with both years but not interactive
#see if this can be done with polar plots
x1 = df_covis_rc2010c['azimuth']
x2 = df_covis_rc2011c['azimuth']

colors = ['blue', 'orange']
names = ['2010', '2011']

plt.hist([x1, x2], bins = int(360/15), density=True,
         color = colors, label=names)

plt.legend()
plt.xlabel('Azimuth')
plt.ylabel('Normalized Frequency')
llama = plt.show()
llama

In [ ]:
#interactive histograms but separate
(df_covis_rc2010c.hvplot.hist(y='azimuth') + df_covis_rc2011c.hvplot.hist(y='azimuth')).cols(1)

**Inclination**

In [ ]:
#histogram with both years but not interactive

x1 = df_covis_rc2010c['inclination']
x2 = df_covis_rc2011c['inclination']

colors = ['blue', 'orange']
names = ['2010', '2011']

plt.hist([x1, x2], bins = int(360/15), density=True,
         color = colors, label=names)

plt.legend()
plt.xlabel('Inclination')
plt.ylabel('Normalized Frequency')
plt.title('2010 and 2011 Inclination')
plt.savefig('HistInc.png')
plt.show()

In [ ]:
#interactive histograms but separate
(df_covis_rc2010c.hvplot.hist(y='inclination') + df_covis_rc2011c.hvplot.hist(y='inclination')).cols(1)

## 2010

**C46036**

Azimuth

In [ ]:
df_covis_rc2010c.hvplot.scatter(y='azimuth',color='inclination',
                                     cmap='viridis', s= 'wnd_spd',
                                     scale = 2).hist()

Inclination

In [ ]:
df_covis_rc2010c.hvplot.scatter(y='inclination',color='inclination',
                                     cmap='viridis', s= 'wnd_spd',
                                     scale = 2).hist()

**Tillamook**

Azimuth

In [ ]:
df_covis_rc2010T.hvplot.scatter(y='azimuth',color='inclination',
                                     cmap='viridis', s= 'wnd_spd',
                                     scale = 2).hist()

Inclination

In [ ]:
df_covis_rc2010T.hvplot.scatter(y='inclination',color='inclination',
                                     cmap='viridis', s= 'wnd_spd',
                                     scale = 2).hist()

## 2011

**C46036**

Azimuth

In [ ]:
df_covis_rc2011c.hvplot.scatter(y='azimuth',color='inclination',
                                     cmap='viridis', s= 'wnd_spd',
                                     scale = 2).hist()

Inclination

In [ ]:
df_covis_rc2011c.hvplot.scatter(y='inclination',color='inclination',
                                     cmap='viridis', s= 'wnd_spd',
                                     scale = 2).hist()

**Tillamook**

Azimuth

In [ ]:
df_covis_rc2011T.hvplot.scatter(y='azimuth',color='inclination',
                                     cmap='viridis', s= 'wnd_spd',
                                     scale = 2).hist()

Inclination

In [ ]:
df_covis_rc2011T.hvplot.scatter(y='inclination',color='inclination',
                                     cmap='viridis', s= 'wnd_spd',
                                     scale = 2).hist()

**Cross Correlation Test**

In [ ]:
plt.xcorr(df_covis_rc2010c['azimuth'], df_covis_rc2010c['inclination'], normed=True, usevlines=True, maxlags=20, data=None)